### This cell import the necessary library

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from camutils import Camera,triangulate
from meshutils import writeply
import pickle
import cv2
from skimage import io
from scipy.spatial import Delaunay
from collections import defaultdict
import triangle as tr

# smoothing function
# from decodeims import smooth

# this function handle pruning and smoothing
# from decodeims import processMesh

### calibrate cameras with extrinsics.py
 getCameras uses the intrinsic parameters from calibrate.py, along the various functions
 and returns two camera (Left, Right) with the calibrated extrinsic paramaters

In [7]:
from extrinsics import getCameras
camL, camR = getCameras()

### This helper function convert a graycode string to a decimal string

In [ ]:
def gray_to_num(gray_string):
    out = gray_string[0]
    
    for i in range(1, len(gray_string)):
        if gray_string[i] == out[-1]:
            out += '0'
        else:
            out += '1'
    return int(out,2)

### Given a path to the colored images, isolate the background with a threshold and return a foreground mask

In [ ]:
def getForegroundMask(imprefix_color, threshold):
    
    fore = io.imread(imprefix_color + '01.png', as_gray=True)        
    if (fore.dtype == np.uint8):
        fore = fore.astype(float) / 256
        
    back = io.imread(imprefix_color + '00.png', as_gray=True)        
    if (back.dtype == np.uint8):
        back = back.astype(float) / 256
        
    mask = np.zeros(fore.shape)
        
    diff = fore - back
    mask[np.absolute(diff) > threshold] = 1
    
    return mask

### Given the pts2 and the colored image, return a 3xN array containing color value at each pixel

In [ ]:
def coorToColor(imprefix_color, pts2):
    im = io.imread(imprefix_color + '01.png', as_gray=False)        
    if (im.dtype == np.uint8):
        im = im.astype(float) / 256
        
    color = np.zeros((3,pts2.shape[1]))
    
    for i in range(pts2.shape[1]):
        pixel = im[pts2[1, i], pts2[0, i]]
        color[0, i] = pixel[0]
        color[1, i] = pixel[1]
        color[2, i] = pixel[2]
    
    return color

### Given pts3 and Delaunay simplices, average each point's position with its neighbors. This function essentially smooth the points

In [ ]:
def smooth(points,triangles):
    points = points.T
    out = np.zeros(points.shape)

    neighbors = makeNeighborDict(triangles)
    
    for i in range(len(points)):
        connected = np.array(list(neighbors[i])).astype(int)

        mean = np.mean(points[connected],axis=0)
        out[i] = mean
        
    return out.T

### Given a Delaunay simplices array, create a dictionary where every point's index is paired with a list of the indices of its neighbor. This is used for smoothing

In [ ]:
def makeNeighborDict(tri):
    out = defaultdict(set)
    for face in tri:
        for vertex in face: # nested loop is fine here since this loops a max of 3
            for other_vertex in face: # again this is fine since there's only 2 other vertices in the face (please forgive me) 
                if vertex != other_vertex:  # a vertex cant be its own neighbor
                    out[vertex].add(other_vertex)
    return out

### For box-pruning, given a point and the limits, return False if the point is out of bound, and true otherwise

In [ ]:
def check_bound(point, boxlimits):
    x = point[0]
    y = point[1]
    z = point[2]
    
    if x < boxlimits[0] or x > boxlimits[1]:
        return True
    elif y < boxlimits[2] or y > boxlimits[3]:
        return True
    elif z < boxlimits[4] or z > boxlimits[5]:
        return True
    
    return False

### Decode the structured illumination images and give each pixel a unique ID

In [2]:
def decode(imprefix,start,threshold):
    """
    Given a sequence of 20 images of a scene showing projected 10 bit gray code, 
    decode the binary sequence into a decimal value in (0,1023) for each pixel.
    Mark those pixels whose code is likely to be incorrect based on the user 
    provided threshold.  Images are assumed to be named "imageprefixN.png" where
    N is a 2 digit index (e.g., "img00.png,img01.png,img02.png...")
 
    Parameters
    ----------
    imprefix : str
       Image name prefix
      
    start : int
       Starting index
       
    threshold : float
       Threshold to determine if a bit is decodeable
       
    Returns
    -------
    code : 2D numpy.array (dtype=float)
        Array the same size as input images with entries in (0..1023)
        
    mask : 2D numpy.array (dtype=logical)
        Array indicating which pixels were correctly decoded based on the threshold
    
    """
    
    # we will assume a 10 bit code
    nbits = 10
    
    # don't forget to convert images to grayscale / float after loading them in
    imnames = np.arange(start,start+20)
    imnames = [imprefix + str(num).zfill(2) + '.png' for num in imnames]
    imgs = list()
    for fname in imnames:          
        I = io.imread(fname, as_gray=True)        
        
        #convert to float data type if necessary
        if (I.dtype == np.uint8):
            I = I.astype(float) / 256                
    
        #finally, store the array in our list of images
        imgs.append(I)                
    
    mask = np.ones(imgs[0].shape)
    
    binary_imgs = list()
    
    for i in range(0, len(imgs), 2):
        first = imgs[i]
        second = imgs[i+1]

        diff = first - second
        binary = np.where(diff > 0, 1, 0)
        thresh = threshold
        mask[np.absolute(diff) < thresh] = 0
        binary_imgs.append(binary)
        
    final_string = np.copy(binary_imgs[0].astype(str))
    
    for i in range(1, len(binary_imgs)):        
        final_string = np.core.defchararray.add(final_string,binary_imgs[i].astype(str))   
        
        
    final_string = np.vectorize(gray_to_num)(final_string)
    
    code = final_string
    
    return code,mask

### This function reconstructs by matching the pixels from the left and right images and triangulate them to produce pts3. The color data is also averaged for each pixel and returned

In [3]:
def reconstruct(imprefixL,imprefixR,threshold,camL,camR):
    # Decode the H and V coordinates for the two views
    LV,LVmask = decode(imprefixL,0,threshold)
    LH,LHmask = decode(imprefixL,20,threshold)
    foreL = getForegroundMask(imprefixL[:-9] + 'color_C0_', 0.1)
    
    
    RV,RVmask = decode(imprefixR,0,threshold)
    RH,RHmask = decode(imprefixR,20,threshold)
    foreR = getForegroundMask(imprefixL[:-9] + 'color_C1_', 0.1)

    # Construct the combined 20 bit code C = H + 1024*V and mask for each view
    Lmask = LVmask * LHmask * foreL
    Rmask = RVmask * RHmask * foreR
    
    Left = LH + 1024 * LV * Lmask
    Right = RH + 1024 * RV * Rmask
    
    # Find the indices of pixels in the left and right code image that 
    # have matching codes. If there are multiple matches, just
    # choose one arbitrarily.
    lr, matchL, matchR = np.intersect1d(Left, Right, return_indices=True)
    
    CL = Left.flatten()
    CR = Right.flatten()
    
    # Let CL and CR be the flattened arrays of codes for the left and right view
    # Suppose you have computed arrays of indices matchL and matchR so that 
    # CL[matchL[i]] == CR[matchR[i]] for all i.  The code below gives one approach
    # to generating the corresponding pixel coordinates for the matched pixels.
    w = Left.shape[1]
    h = Left.shape[0]
    
    xx,yy = np.meshgrid(range(w),range(h))
    xx = np.reshape(xx,(-1,1))
    yy = np.reshape(yy,(-1,1))
    pts2R = np.concatenate((xx[matchR].T,yy[matchR].T),axis=0)
    pts2L = np.concatenate((xx[matchL].T,yy[matchL].T),axis=0)

    # Now triangulate the points
    pts3 = triangulate(pts2L,camL, pts2R,camR)
    
    # we want to average the color value between the left and right image, then save the colors information as well
    colorLeft = coorToColor(imprefixL[:-9] + 'color_C0_', pts2L)
    colorRight = coorToColor(imprefixL[:-9] + 'color_C1_', pts2R)
    colors = np.mean([colorLeft,colorRight],axis=0)
    
    return pts2L,pts2R,pts3,colors

### This function applies box and triangle pruning, as well as remapping the indices of the Delaunay triangulation after points had been pruned

In [ ]:
def processMesh(t_pts3, t_pts2L, t_pts2R, t_colors):

    boxlimits = np.array([-200,200,-2,18,-30,-0])
    trithresh = 1
    
    # triangulate the 2D points to get the surface mesh
    tri = tr.delaunay(t_pts2L.T)

    bad_face = list()
    for i in range(len(tri)):
        triangle = tri[i]
        a = t_pts3.T[triangle[0]]
        b = t_pts3.T[triangle[1]]
        c = t_pts3.T[triangle[2]]
        
        ab = np.linalg.norm(a-b)
        bc = np.linalg.norm(c-b)
        ac = np.linalg.norm(a-c)
        
        max = np.amax([ab,bc,ac])
        if max > trithresh:
            bad_face.append(i)
        elif check_bound(a,boxlimits) or check_bound(b,boxlimits) or check_bound(c,boxlimits):
            bad_face.append(i)
            
            
    tri = np.delete(tri,bad_face,0)

    bad_ind = np.arange(0,t_pts3.shape[1])
    
    bad_ind = np.setdiff1d(bad_ind, tri.flatten())
    
    map = np.ones(t_pts3.shape[1]) * -1  
    
    bad_ind = np.unique(bad_ind)
    
    t_pts3 = np.delete(t_pts3,bad_ind,1)
    t_pts2L = np.delete(t_pts2L,bad_ind,1)
    t_pts2R = np.delete(t_pts2R,bad_ind,1)
    t_colors = np.delete(t_colors,bad_ind,1)
  
    tokeep = np.unique(tri.flatten())
    map[tokeep] = np.arange(0,tokeep.shape[0])
    tri = map[tri]
    
    t_pts3 = smooth(t_pts3,tri)
    t_pts3 = smooth(t_pts3,tri)
    

    return t_pts3, t_pts2L, t_pts2R, t_colors, tri

### Given a path and subfolder number, this function call reconstruct on the input images within said folder

In [ ]:
def compute(OBJECT, take):
    imprefixC0 = OBJECT + '/grab_' + str(take) + '_u/frame_C0_'
    imprefixC1 = OBJECT + '/grab_' + str(take) + '_u/frame_C1_'
    threshold = 0.02
    pts2L,pts2R,pts3,colors = reconstruct(imprefixC0,imprefixC1,threshold,camL,camR)
    
    
    return pts2L,pts2R,pts3,colors

### This function applied ProcessMesh (pruning) to reconstructed data and optionally save it to ply or pickle file

In [4]:
def processAndSave(pts2L,pts2R,pts3,colors,save_file, write_ply, OBJECT, take):
    pts3,pts2L,pts2R,colors,tri = processMesh(pts3,pts2L,pts2R,colors)
    
    if save_file:
        output = {}
        output['pts2L'] = pts2L
        output['pts2R'] = pts2R
        output['pts3'] = pts3
        output['colors'] = colors
        output['tri'] = tri

        with open('reconstructed_' + OBJECT + '_' + str(take) + '.pkl', 'wb') as out:
            pickle.dump(output, out)
            
    if write_ply:
        writeply(pts3,colors,tri,OBJECT + '_' + str(take) + '.ply')
    
    return pts2L,pts2R,pts3,colors,tri

### This loop through each subfolder of and object and process the input images set within

In [ ]:
for i in range(1,5):
    pts2L,pts2R,pts3,colors = compute("manny",i)
    f_pts2L,f_pts2R,f_pts3,f_colors,tri = processAndSave(pts2L,pts2R,pts3,colors,False, True, 'manny', i)

### This code display the output for debugging purpose

In [5]:
import trimesh
mesh = trimesh.Trimesh(vertices=f_pts3.T,faces=tri[:,[0,2,1]])
mesh.show()
